# Basket Size 예측

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
products = pd.read_csv("data/products.csv")
aisles = pd.read_csv("data/aisles.csv")
departments = pd.read_csv("data/departments.csv")
orders = pd.read_csv("data/orders.csv")
prior = pd.read_csv("data/order_products__prior.csv")
train = pd.read_csv("data/order_products__train.csv")

In [ ]:
priorXorders = prior.merge(orders, on='order_id')
priorXorders.head()

## prior의 basket size

In [ ]:
prior_order_basketsize = pd.DataFrame()
prior_order_basketsize['basketsize'] = prior.groupby('order_id').size()
prior_order_basketsize

## train의 basketsize

In [ ]:
train_order_basketsize = pd.DataFrame()
train_order_basketsize['basketsize'] = train.groupby('order_id').size()
train_order_basketsize

In [ ]:
order_basketsize = prior_order_basketsize.append(train_order_basketsize)

# order와 basketsize를 합침

In [ ]:
order_plus_basketsize = orders.join(order_basketsize, on = ['order_id'])
order_plus_basketsize

In [ ]:
prior_basket_Info = order_plus_basketsize[order_plus_basketsize['eval_set'] == 'prior']
prior_basket_Info

In [ ]:
train_basket_Info = order_plus_basketsize[order_plus_basketsize['eval_set'] == 'train']
train_basket_Info

In [ ]:
average_basketsize = pd.DataFrame()
average_basketsize['user_avg_basketsize'] = prior_basket_Info.groupby('user_id')['basketsize'].mean()
Average_Basketsize = pd.concat([average_basketsize, train_basket_Info], axis=1, join='inner')
del average_basketsize

Average_Basketsize = Average_Basketsize.drop(['eval_set','order_number','order_dow','order_hour_of_day','days_since_prior_order'], 1)
Average_Basketsize = Average_Basketsize.set_index('user_id')
Average_Basketsize

In [ ]:
import math
mse = 0.0
for row in Average_Basketsize.itertuples():
    a = row.user_avg_basketsize
    b = row.basketsize
    mse += (a-b)*(a-b)
N = len(Average_Basketsize.index)
mse /= N
rmse = math.sqrt(mse)

print('mse : ',mse)
print('rmse : ',rmse)